## Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df=application_df.drop(['EIN', 'NAME'],1)

In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
app_counts=application_df['APPLICATION_TYPE'].value_counts()
app_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace= list(app_counts[app_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df['CLASSIFICATION'].value_counts()

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts[class_counts > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1237        9
C1235        9
C7210        7
C4100        6
C1720        6
C2400        6
C1600        5
C1257        5
C2710        3
C0           3
C1260        3
C1267        2
C3200        2
C1256        2
C1246        2
C1234        2
Name: CLASSIFICATION, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace= list(class_counts[class_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
df = pd.get_dummies(application_df)

In [11]:
# Split our preprocessed data into our features and target arrays
X = df.drop(['IS_SUCCESSFUL'], axis='columns').values
y = df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test= train_test_split(X, y, random_state=42)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation='relu',input_dim=len(X_train[0])))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=15, activation='relu'))
# Third hidden layer
nn.add(tf.keras.layers.Dense(units=5, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))


# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                1320      
_________________________________________________________________
dense_1 (Dense)              (None, 15)                465       
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 80        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 6         
Total params: 1,871
Trainable params: 1,871
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6095 - accuracy: 0.6973
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5544 - accuracy: 0.7305
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5541 - accuracy: 0.7253
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5494 - accuracy: 0.7325
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5493 - accuracy: 0.7337
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5466 - accuracy: 0.7328
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5436 - accuracy: 0.7358
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5449 - accuracy: 0.7344
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5436 - accuracy: 0.7337
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5459 - accura

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5585 - accuracy: 0.7278
Loss: 0.5585348010063171, Accuracy: 0.7278134226799011


In [20]:
# Train the model
fit_model1 = nn.fit(X_train_scaled,y_train, epochs=150)

Epoch 1/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5354 - accuracy: 0.7398
Epoch 2/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5353 - accuracy: 0.7405
Epoch 3/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5354 - accuracy: 0.7393
Epoch 4/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5357 - accuracy: 0.7390
Epoch 5/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5355 - accuracy: 0.7396
Epoch 6/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5352 - accuracy: 0.7391
Epoch 7/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5353 - accuracy: 0.7399
Epoch 8/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5352 - accuracy: 0.7400
Epoch 9/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5354 - accuracy: 0.7398
Epoch 10/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5352 - accura

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5662 - accuracy: 0.7275
Loss: 0.5662457346916199, Accuracy: 0.7274635434150696


In [19]:
# Export our model to HDF5 file
#nn.save("AlphabetSoupCharity_Optimization.h5")